# GOAL : 
## FIND THE TYPE OF CUSTOMERS WHO PURCHASE CHIPS AND THEIR PURCHASING BEHAVIOUR WITHIN THE REGION

# PART ONE

## Examine Transaction Data
1. Look for inconsistencies, missing data across the dataset, outliers, correctly identified category items, numeric data across all tables.
2. In case of identified anomalies, make necessary changes to the dataset and save it.

In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chardet

In [2]:
# encoding used in the file
with open ("QVI_transaction_data.csv", mode='rb') as file:
    raw_bytes = file.read(5)
    detected_encoding = chardet.detect(raw_bytes)['encoding']
    print(detected_encoding)

ascii


In [3]:
# convert from ascii to utf-8
with open("QVI_transaction_data.csv", encoding='utf-8') as file:
    rows = list(csv.reader(file))
    header = rows[0]
    print(rows[:10])

[['DATE', 'STORE_NBR', 'LYLTY_CARD_NBR', 'TXN_ID', 'PROD_NBR', 'PROD_NAME', 'PROD_QTY', 'TOT_SALES'], ['43390', '1', '1000', '1', '5', 'Natural Chip        Compny SeaSalt175g', '2', '6'], ['43599', '1', '1307', '348', '66', 'CCs Nacho Cheese    175g', '3', '6.3'], ['43605', '1', '1343', '383', '61', 'Smiths Crinkle Cut  Chips Chicken 170g', '2', '2.9'], ['43329', '2', '2373', '974', '69', 'Smiths Chip Thinly  S/Cream&Onion 175g', '5', '15'], ['43330', '2', '2426', '1038', '108', 'Kettle Tortilla ChpsHny&Jlpno Chili 150g', '3', '13.8'], ['43604', '4', '4074', '2982', '57', 'Old El Paso Salsa   Dip Tomato Mild 300g', '1', '5.1'], ['43601', '4', '4149', '3333', '16', 'Smiths Crinkle Chips Salt & Vinegar 330g', '1', '5.7'], ['43601', '4', '4196', '3539', '24', 'Grain Waves         Sweet Chilli 210g', '1', '3.6'], ['43332', '5', '5026', '4525', '42', 'Doritos Corn Chip Mexican Jalapeno 150g', '1', '3.9']]


# INSIGHTS

1. **ASCII** encoding is used in the csv file.
2. The Date Column doers not look like a date.
3. Numbers are represented as strings. 
4. Format headers to have the correct spelling and start with uppercase only.
5. Separate quantity from product name 

In [4]:
# explore the dataset
def explore_dataset(dataset,start,end,rows_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print("\n")
    if rows_columns:
        print("There are {} rows".format(len(dataset)))
        print("There are {} columns".format(len(dataset[0])))
    

In [5]:
explore_dataset(rows,0,10,True)

['DATE', 'STORE_NBR', 'LYLTY_CARD_NBR', 'TXN_ID', 'PROD_NBR', 'PROD_NAME', 'PROD_QTY', 'TOT_SALES']


['43390', '1', '1000', '1', '5', 'Natural Chip        Compny SeaSalt175g', '2', '6']


['43599', '1', '1307', '348', '66', 'CCs Nacho Cheese    175g', '3', '6.3']


['43605', '1', '1343', '383', '61', 'Smiths Crinkle Cut  Chips Chicken 170g', '2', '2.9']


['43329', '2', '2373', '974', '69', 'Smiths Chip Thinly  S/Cream&Onion 175g', '5', '15']


['43330', '2', '2426', '1038', '108', 'Kettle Tortilla ChpsHny&Jlpno Chili 150g', '3', '13.8']


['43604', '4', '4074', '2982', '57', 'Old El Paso Salsa   Dip Tomato Mild 300g', '1', '5.1']


['43601', '4', '4149', '3333', '16', 'Smiths Crinkle Chips Salt & Vinegar 330g', '1', '5.7']


['43601', '4', '4196', '3539', '24', 'Grain Waves         Sweet Chilli 210g', '1', '3.6']


['43332', '5', '5026', '4525', '42', 'Doritos Corn Chip Mexican Jalapeno 150g', '1', '3.9']


There are 264837 rows
There are 8 columns


In [6]:
# check if there are empty sublists in our list
rows =  [sublist for sublist in rows if sublist]
explore_dataset(rows,0,5,True)

['DATE', 'STORE_NBR', 'LYLTY_CARD_NBR', 'TXN_ID', 'PROD_NBR', 'PROD_NAME', 'PROD_QTY', 'TOT_SALES']


['43390', '1', '1000', '1', '5', 'Natural Chip        Compny SeaSalt175g', '2', '6']


['43599', '1', '1307', '348', '66', 'CCs Nacho Cheese    175g', '3', '6.3']


['43605', '1', '1343', '383', '61', 'Smiths Crinkle Cut  Chips Chicken 170g', '2', '2.9']


['43329', '2', '2373', '974', '69', 'Smiths Chip Thinly  S/Cream&Onion 175g', '5', '15']


There are 264837 rows
There are 8 columns


In [7]:
# check for duplicates
duplicate_entries = []
unique_entries = set()
rows_with_duplicates = []
for row in rows:
    transaction_id = row[3]
    if transaction_id not in unique_entries:
        unique_entries.add(transaction_id)
    else:
        duplicate_entries.append(transaction_id)
        rows_with_duplicates.append(row)
print("There are {} duplicate entries".format(len(duplicate_entries)))
print("There are {} unique entries".format(len(unique_entries)))
print(rows_with_duplicates[:10])

There are 1709 duplicate entries
There are 263128 unique entries
[['43605', '55', '55073', '48887', '113', 'Twisties Chicken270g', '1', '4.6'], ['43475', '7', '7364', '7739', '20', 'Doritos Cheese      Supreme 330g', '2', '11.4'], ['43391', '12', '12301', '10982', '93', 'Doritos Corn Chip Southern Chicken 150g', '2', '7.8'], ['43351', '16', '16427', '14546', '81', 'Pringles Original   Crisps 134g', '1', '3.7'], ['43315', '19', '19272', '16683', '31', 'Infzns Crn Crnchers Tangy Gcamole 110g', '2', '7.6'], ['43401', '47', '47204', '42616', '45', 'Smiths Thinly Cut   Roast Chicken 175g', '2', '6'], ['43609', '48', '48179', '44177', '56', 'Cheezels Cheese Box 125g', '2', '4.2'], ['43559', '55', '55036', '48663', '91', 'CCs Tasty Cheese    175g', '2', '4.2'], ['43282', '55', '55073', '48884', '91', 'CCs Tasty Cheese    175g', '2', '4.2'], ['43489', '58', '58121', '53351', '42', 'Doritos Corn Chip Mexican Jalapeno 150g', '2', '7.8']]


Out of these duplicate entries, they have similar transaction ids, loyalty card numbers but different products. so we will leave them as they are.

In [8]:
# get the index of every column
col_index = {}
for i in range(len(header)):
    col_index[header[i]] = i
print(col_index)

{'DATE': 0, 'STORE_NBR': 1, 'LYLTY_CARD_NBR': 2, 'TXN_ID': 3, 'PROD_NBR': 4, 'PROD_NAME': 5, 'PROD_QTY': 6, 'TOT_SALES': 7}


In [ ]:
from openpyxl import load_workbook
from datetime import datetime, timedelta

def excel_serial_date_to_datetime(serial_date):
    base_date = datetime(1899, 12, 31)
    delta = timedelta(days=serial_date)
    resulting_date = base_date + delta
    return resulting_date

# Example usage
excel_file_path = 'C:/Users/user/I am learning ML/Quantium VIrtual Experience/Data Preparation and Customer Analysis/QVI_transaction_data.xlsx'
sheet_name = 'in'
date_column_index = 0  # Adjust the column index to match the column containing Excel serial dates

# Load the Excel file
workbook = load_workbook(excel_file_path)

# Select the desired sheet
sheet = workbook[sheet_name]

# Iterate over the rows and convert Excel serial dates to datetime objects
for row in sheet.iter_rows(min_row=2, values_only=True):  # Assuming the first row is a header
    serial_date = row[date_column_index]
    date = excel_serial_date_to_datetime(serial_date)
    print(date[:5])


2018-10-18 00:00:00
2019-05-15 00:00:00
2019-05-21 00:00:00
2018-08-18 00:00:00
2018-08-19 00:00:00
2019-05-20 00:00:00
2019-05-17 00:00:00
2019-05-17 00:00:00
2018-08-21 00:00:00
2018-08-19 00:00:00
2019-05-18 00:00:00
2018-08-21 00:00:00
2019-05-19 00:00:00
2018-08-18 00:00:00
2019-05-16 00:00:00
2019-05-20 00:00:00
2018-08-19 00:00:00
2018-08-15 00:00:00
2018-08-18 00:00:00
2019-05-17 00:00:00
2019-05-20 00:00:00
2018-08-17 00:00:00
2018-08-17 00:00:00
2018-08-20 00:00:00
2018-08-16 00:00:00
2019-05-16 00:00:00
2018-08-20 00:00:00
2019-05-16 00:00:00
2018-08-16 00:00:00
2019-05-21 00:00:00
2019-05-17 00:00:00
2019-05-16 00:00:00
2019-05-21 00:00:00
2019-05-19 00:00:00
2018-08-17 00:00:00
2018-08-20 00:00:00
2018-08-19 00:00:00
2018-08-18 00:00:00
2018-08-19 00:00:00
2018-08-19 00:00:00
2019-05-15 00:00:00
2019-05-21 00:00:00
2019-05-21 00:00:00
2019-05-17 00:00:00
2018-08-19 00:00:00
2019-05-17 00:00:00
2019-05-17 00:00:00
2018-08-17 00:00:00
2019-05-16 00:00:00
2019-05-15 00:00:00
